In [20]:
import pandas as pd
import yfinance as yf
from sqlalchemy import create_engine

print("Notebook ready")


Notebook ready


In [21]:
import pandas as pd
import yfinance as yf
from sqlalchemy import create_engine
from fredapi import Fred


In [22]:
db_engine = create_engine("sqlite:///../data/equity_pipeline.db")

print("Database connected")


Database connected


In [23]:
stock_list = ["SPY", "AAPL", "MSFT"]

stock_data = yf.download(stock_list,start="2015-01-01",auto_adjust=True,progress=False)
stock_prices = stock_data["Close"].reset_index()

stock_prices.head()


Ticker,Date,AAPL,MSFT,SPY
0,2015-01-02,24.237560,39.858456,170.589630
1,2015-01-05,23.554743,39.491936,167.508789
2,2015-01-06,23.556953,38.912289,165.931076
3,2015-01-07,23.887274,39.406693,167.998734
4,2015-01-08,24.805080,40.565948,170.979889


In [24]:
stock_prices.to_sql("raw_prices",db_engine,if_exists="replace",index=False)

print("Stock price data saved to SQL database")


Stock price data saved to SQL database


In [25]:
# Step 1: Create FRED client (replace with your real key)
fred_key = "f75b2a9f91c67d406bcf73ddc4f6321f"
fred = Fred(api_key=fred_key)

print("FRED connected")


FRED connected


In [26]:
# Step 2: Download macroeconomic data (properly aligned)

cpi = fred.get_series("CPIAUCSL")
unemployment = fred.get_series("UNRATE")
interest_rate = fred.get_series("FEDFUNDS")
vix = fred.get_series("VIXCLS")

macro_data = pd.concat(
    [cpi, unemployment, interest_rate, vix],
    axis=1
)

macro_data.columns = ["cpi", "unemployment", "interest_rate", "vix"]
macro_data = macro_data.reset_index()
macro_data.rename(columns={"index": "date"}, inplace=True)

print(macro_data)


            date    cpi  unemployment  interest_rate    vix
0     1947-01-01  21.48           NaN            NaN    NaN
1     1947-02-01  21.62           NaN            NaN    NaN
2     1947-03-01  22.00           NaN            NaN    NaN
3     1947-04-01  22.00           NaN            NaN    NaN
4     1947-05-01  21.95           NaN            NaN    NaN
...          ...    ...           ...            ...    ...
10038 2026-01-09    NaN           NaN            NaN  14.49
10039 2026-01-12    NaN           NaN            NaN  15.12
10040 2026-01-13    NaN           NaN            NaN  15.98
10041 2026-01-14    NaN           NaN            NaN  16.75
10042 2026-01-15    NaN           NaN            NaN  15.84

[10043 rows x 5 columns]


In [27]:
# this is to check for alignment issues and if we have murged the data correctly 
macro_data.notna().sum()

date             10043
cpi                947
unemployment       935
interest_rate      858
vix               9102
dtype: int64

In [30]:
# Step 3: Save macro data to SQL (THIS WAS MISSING)

macro_data.to_sql(
    "raw_macro",
    db_engine,
    if_exists="replace",
    index=False
)

print("raw_macro saved to database")


raw_macro saved to database


In [32]:
pd.read_sql(
    "SELECT name FROM sqlite_master WHERE type='table'",
    db_engine
)


,name
0,raw_prices
1,raw_macro
2,monthly_prices
3,monthly_returns
4,monthly_macro
5,final_dataset


In [33]:
from sqlalchemy import text

sql_file = open("../sql/01_cleaning.sql").read()

with db_engine.connect() as conn:
    for statement in sql_file.split(";"):
        if statement.strip():
            conn.execute(text(statement))

print("Phase 2 SQL executed successfully")

Phase 2 SQL executed successfully
